In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["abalone_input.pd","abalone_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["cars_input.pd","cars_target.pd"], 
    ["students_input.pd","students_target.pd"],
    ["adult_input.pd","adult_target.pd"], # Large
    ["bank_input.pd","bank_target.pd"], # Large
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["occupancy_input.pd","occupancy_target.pd"], # Easy
    ["pdm6_input.pd","pdm6_target.pd"], # Easy
    ["banknote_input.pd","banknote_target.pd"], # Easy
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 3,
    "iterations": 100,
    "max_depth": 2,
    "init_learning_rate": 10,
    "learning_rate_decay": 0.995,
    "initialization_method": "to be overwritten",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "to be overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": False,
    "use_posterior": False,
    }

In [4]:
gate_reduction_methods = [
    "feature_importance_lr_max",
    None
]

In [5]:
initialization_methods = [
    Random_init(seed=None),
    Kmeans_init(),
    KDTmeans_init(alpha=2.5, beta=0.25, gamma=0.3),
    BGM_init(n_components=parameters["n_experts"],
             mean_precision_prior=0.35,
             weight_concentration_prior_type="dirichlet_process",
             weight_concentration_prior=0.5,
             weight_cutoff=0.0)    
]

In [6]:
start = timer()
runs = 2
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
    dict_results = {
        "dataset" : dataset[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in gate_reduction_methods:
        for init_method in initialization_methods:
            init_method_name = init_method.__class__.__name__
            
            print("Starting",dataset[0],method,init_method_name,"...")
            parameters["use_2_dim_gate_based_on"] = method
            if parameters["use_2_dim_gate_based_on"] is not None:
                parameters["use_2_dim_clustering"] = True
            else:
                parameters["use_2_dim_clustering"] = False
            parameters["initialization_method"] = init_method

            train_accuracies = []
            val_accuracies = []
            train_accuracies_i = [[] for i in range(parameters["iterations"])]
            
            rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
            for train_idx, val_idx in rkf.split(data_input):
                if use_dataframe:
                    X_temp = data_input.iloc[train_idx]
                    y_temp = data_target.iloc[train_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[train_idx]
                    y_temp = data_target[train_idx]

                parameters["X"] = X_temp
                parameters["y"] = y_temp

                modt = MoDT(**parameters)
                modt.fit(**parameters_fit)
                train_accuracies.append(modt.score_internal_disjoint())

                if use_dataframe:
                    X_temp = data_input.iloc[val_idx]
                    y_temp = data_target.iloc[val_idx]
                    X_temp.reset_index(inplace=True, drop=True)
                    y_temp.reset_index(inplace=True, drop=True)
                else:
                    X_temp = data_input[val_idx]
                    y_temp = data_target[val_idx]
                val_accuracies.append(modt.score(X_temp, y_temp))
                for i in range(parameters["iterations"]):
                    train_accuracies_i[i].append(modt.all_accuracies[i])

            train_accuracy = np.mean(train_accuracies)
            val_accuracy = np.mean(val_accuracies)
            train_std = np.std(train_accuracies)
            val_std = np.std(val_accuracies)
            dict_results[str(method) + "_" + str(init_method_name) + "_train"] = train_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_test"] = val_accuracy
            dict_results[str(method) + "_" + str(init_method_name) + "_train_std"] = train_std
            dict_results[str(method) + "_" + str(init_method_name) + "_test_std"] = val_std                        

            for i in range(parameters["iterations"]):
                train_i_acc = np.mean(train_accuracies_i[i])
                dict_results[str(method) + "_" + str(init_method_name) + "_train_i_" + str(i)] = modt.all_accuracies[i]
        
    rows.append(dict_results)
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting breast_cancer_input.np ...
Starting breast_cancer_input.np feature_importance_lr_max Random_init ...
Starting breast_cancer_input.np feature_importance_lr_max Kmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max KDTmeans_init ...
Starting breast_cancer_input.np feature_importance_lr_max BGM_init ...
Starting breast_cancer_input.np None Random_init ...
Starting breast_cancer_input.np None Kmeans_init ...
Starting breast_cancer_input.np None KDTmeans_init ...
Starting breast_cancer_input.np None BGM_init ...
Starting iris_input.pd ...
Starting iris_input.pd feature_importance_lr_max Random_init ...
Starting iris_input.pd feature_importance_lr_max Kmeans_init ...
Starting iris_input.pd feature_importance_lr_max KDTmeans_init ...
Starting iris_input.pd feature_importance_lr_max BGM_init ...
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Separation unsuccessful. Gate initialized randomly.
Starting iris_in

In [7]:
df_performance

,dataset,n_features,feature_importance_lr_max_Random_init_train,feature_importance_lr_max_Random_init_test,feature_importance_lr_max_Random_init_train_std,feature_importance_lr_max_Random_init_test_std,feature_importance_lr_max_Random_init_train_i_0,feature_importance_lr_max_Random_init_train_i_1,feature_importance_lr_max_Random_init_train_i_2,feature_importance_lr_max_Random_init_train_i_3,...,None_BGM_init_train_i_90,None_BGM_init_train_i_91,None_BGM_init_train_i_92,None_BGM_init_train_i_93,None_BGM_init_train_i_94,None_BGM_init_train_i_95,None_BGM_init_train_i_96,None_BGM_init_train_i_97,None_BGM_init_train_i_98,None_BGM_init_train_i_99
0,breast_cancer_input.np,10,0.969539,0.927965,0.002605,0.024957,0.950820,0.957845,0.964871,0.957845,...,0.983607,0.983607,0.983607,0.983607,0.983607,0.983607,0.983607,0.983607,0.983607,0.983607
1,iris_input.pd,4,0.988859,0.946746,0.010709,0.023111,0.964602,0.955752,0.982301,0.991150,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,steel_input.pd,27,0.672251,0.642430,0.041461,0.045991,0.568681,0.573489,0.574176,0.574176,...,0.682692,0.682692,0.682692,0.682692,0.682692,0.682692,0.682692,0.682692,0.682692,0.682692
3,abalone_input.pd,8,0.736573,0.724681,0.004281,0.005607,0.691669,0.696776,0.704756,0.699330,...,0.744654,0.744335,0.744335,0.744335,0.744015,0.743696,0.744015,0.744335,0.743696,0.744335
4,contraceptive_input.pd,9,0.570943,0.543123,0.018035,0.020035,0.501357,0.552941,0.549321,0.551131,...,0.489593,0.489593,0.489593,0.489593,0.489593,0.489593,0.489593,0.489593,0.489593,0.489593


In [8]:
pickle.dump(df_performance, open("dataframes/df_initialization_methods.pd", "wb"))